In [1]:
!pip install catboost xgboost scikit-learn --quiet

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

from google.colab import drive
drive.mount('/content/drive')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.1 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
# === Load & Prepare Data ===
df = pd.read_csv("/content/drive/MyDrive/Network Maintenace - With & without maintenace/encoded-data/Treatment-encoded-dataset-2017-2020/df_encoded_2017_2020.csv")

df = df.dropna().drop_duplicates()

df.drop(['Last_Overlay_Thickness_mean_x'], axis=1, inplace=True)

X = df.drop(columns=['IRI_mean_y'])
y = df[['IRI_mean_y']]

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_x.fit_transform(X_train)
X_val_scaled = scaler_x.transform(X_val)
X_test_scaled = scaler_x.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)


In [3]:
# === Evaluation Function ===
def evaluate_model(model, name):
    # Predictions and true values
    y_pred = scaler_y.inverse_transform(model.predict(X_test_scaled).reshape(-1, 1))
    y_true = y_test.values

    # Metrics calculation
    n = len(y_true)
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    r2 = 1 - (ss_res / ss_tot)
    mae = np.mean(np.abs(y_true - y_pred))
    epsilon = 1e-10
    mape = (100 / n) * np.sum(np.abs((y_true - y_pred) / (y_true + epsilon)))
    vaf = (1 - np.var(y_true - y_pred) / np.var(y_true)) * 100
    rsr = rmse / np.sqrt(np.mean((y_true - np.mean(y_true)) ** 2))
    wmap = (np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))) * 100
    m20 = np.sum(np.abs(y_true - y_pred) <= 0.2 * np.abs(y_true))
    a20index = m20 / n

    # Print results
    print(f"🔹 {name}:")
    print(f"   R²       = {r2:.4f}")
    print(f"   RMSE     = {rmse:.4f}")
    print(f"   MAE      = {mae:.4f}")
    print(f"   MAPE     = {mape:.4f}%")
    print(f"   VAF      = {vaf:.4f}%")
    print(f"   RSR      = {rsr:.4f}")
    print(f"   WMAPE    = {wmap:.4f}%")
    print(f"   a20index = {a20index:.4f}")

    return rmse, r2, mae, mape, vaf, rsr, wmap, a20index

In [4]:
# === Train and Evaluate All Models ===
metrics = {}

In [5]:
# ✅ Random Forest (GridSearchCV)
rf_grid = GridSearchCV(
    RandomForestRegressor(random_state=42),
    {
        'n_estimators': [100, 200],
        'max_features': ['sqrt', 'log2'],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 10],
        'min_samples_leaf': [1, 4]
    },
    cv=5, scoring='r2', n_jobs=-1
)

rf_grid.fit(X_train_scaled, y_train_scaled.ravel())

# ✅ Save metrics
metrics['RF Grid'] = evaluate_model(rf_grid.best_estimator_, "Random Forest (GridSearch)")

# ✅ Print best parameters
print("🔑 Best Parameters for Random Forest (GridSearch):")
for param, value in rf_grid.best_params_.items():
    print(f"   {param}: {value}")


🔹 Random Forest (GridSearch):
   R²       = 0.7586
   RMSE     = 24.9055
   MAE      = 15.8838
   MAPE     = 19.4548%
   VAF      = 75.9484%
   RSR      = 0.4913
   WMAPE    = 19.1622%
   a20index = 0.6688
🔑 Best Parameters for Random Forest (GridSearch):
   max_depth: None
   max_features: sqrt
   min_samples_leaf: 1
   min_samples_split: 2
   n_estimators: 200


In [6]:
# ✅ XGBoost (GridSearchCV)
xgb_grid = GridSearchCV(
    estimator=XGBRegressor(random_state=42, verbosity=0),
    param_grid={
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 6, 10],
        'learning_rate': [0.01, 0.05, 0.1]
    },
    cv=5,
    scoring='r2',
    n_jobs=-1
)

# Fit the model
xgb_grid.fit(X_train_scaled, y_train_scaled.ravel())

# Evaluate the best estimator
metrics['XGB Grid'] = evaluate_model(xgb_grid.best_estimator_, "XGBoost (GridSearch)")

# ✅ Print best parameters and best cross-validation R² score
print("🔑 Best Parameters for XGBoost (GridSearch):")
for param, value in xgb_grid.best_params_.items():
    print(f"   {param}: {value}")

print(f"📈 Best Cross-Validated R² Score: {xgb_grid.best_score_:.4f}")


🔹 XGBoost (GridSearch):
   R²       = 0.7747
   RMSE     = 24.0606
   MAE      = 16.0283
   MAPE     = 20.1947%
   VAF      = 77.5328%
   RSR      = 0.4746
   WMAPE    = 19.3364%
   a20index = 0.6453
🔑 Best Parameters for XGBoost (GridSearch):
   learning_rate: 0.05
   max_depth: 3
   n_estimators: 200
📈 Best Cross-Validated R² Score: 0.7312


In [7]:
# ✅ CatBoost (GridSearchCV)
cat_grid = GridSearchCV(
    estimator=CatBoostRegressor(verbose=0, random_state=42),
    param_grid={
        'iterations': [300, 500, 1000],
        'depth': [6, 8, 10],
        'learning_rate': [0.01, 0.05, 0.1]
    },
    cv=5,
    scoring='r2',
    n_jobs=-1
)

# Fit the model
cat_grid.fit(X_train_scaled, y_train_scaled.ravel())

# Evaluate the best estimator
metrics['Cat Grid'] = evaluate_model(cat_grid.best_estimator_, "CatBoost (GridSearch)")

# ✅ Print best parameters and best cross-validation R² score
print("🔑 Best Parameters for CatBoost (GridSearch):")
for param, value in cat_grid.best_params_.items():
    print(f"   {param}: {value}")

print(f"📈 Best Cross-Validated R² Score: {cat_grid.best_score_:.4f}")


🔹 CatBoost (GridSearch):
   R²       = 0.7896
   RMSE     = 23.2513
   MAE      = 15.3516
   MAPE     = 19.3653%
   VAF      = 79.0288%
   RSR      = 0.4587
   WMAPE    = 18.5202%
   a20index = 0.6688
🔑 Best Parameters for CatBoost (GridSearch):
   depth: 8
   iterations: 1000
   learning_rate: 0.01
📈 Best Cross-Validated R² Score: 0.7391


In [8]:
# === 📊 Tabular Comparison (Updated) ===
df_results = pd.DataFrame([
    {
        "Model": name,
        "R2": r2,
        "RMSE": rmse,
        "MAE": mae,
        "MAPE (%)": mape,
        "VAF (%)": vaf,
        "RSR": rsr,
        "WMAPE (%)": wmape,
        "a20index": a20index
    }
    for name, (rmse, r2, mae, mape, vaf, rsr, wmape, a20index) in metrics.items()
])

# ✅ Round numeric values for better readability
df_results = df_results.round({
    "R2": 4, "RMSE": 4, "MAE": 4, "MAPE (%)": 2,
    "VAF (%)": 2, "RSR": 4, "WMAPE (%)": 2, "a20index": 4
})

# ✅ Sort by R² (Descending)
df_results = df_results.sort_values(by="R2", ascending=False).reset_index(drop=True)

# ✅ Format percentages with % sign
df_results["MAPE (%)"] = df_results["MAPE (%)"].astype(str) + '%'
df_results["VAF (%)"] = df_results["VAF (%)"].astype(str) + '%'
df_results["WMAPE (%)"] = df_results["WMAPE (%)"].astype(str) + '%'

# ✅ Display results
print(f"📊 Results Table Shape: {df_results.shape}")
df_results.head(10)  # or print(df_results.head(10)) if not using Jupyter


📊 Results Table Shape: (3, 9)


Model      R2     RMSE      MAE MAPE (%) VAF (%)     RSR WMAPE (%)  \
0  Cat Grid  0.7896  23.2513  15.3516   19.37%  79.03%  0.4587    18.52%   
1  XGB Grid  0.7747  24.0606  16.0283   20.19%  77.53%  0.4746    19.34%   
2   RF Grid  0.7586  24.9055  15.8838   19.45%  75.95%  0.4913    19.16%   

   a20index  
0    0.6688  
1    0.6453  
2    0.6688

In [9]:
# === 📈 Visual Comparison (Corrected) ===
metrics_list = ["R2", "RMSE", "MAE", "MAPE (%)",
                "VAF (%)", "RSR", "WMAPE (%)", "a20index"]

num_metrics = len(metrics_list)
cols = 2
rows = int(np.ceil(num_metrics / cols))

fig, axes = plt.subplots(rows, cols, figsize=(18, rows * 4))
axes = axes.flatten()  # Flatten in case of multiple rows
colors = sns.color_palette("Set2", n_colors=len(df_results))

for i, metric in enumerate(metrics_list):
    sns.barplot(data=df_results, x="Model", y=metric, ax=axes[i], palette=colors)
    axes[i].set_title(f"{metric} Comparison", fontsize=14)
    axes[i].tick_params(axis='x', rotation=45)
    axes[i].grid(True, linestyle='--', alpha=0.6)

# Hide unused axes if any
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.suptitle("📊 Performance Comparison of ML Models (GridSearch)", fontsize=18)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


/tmp/ipython-input-1770942647.py:14: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(data=df_results, x="Model", y=metric, ax=axes[i], palette=colors)
/tmp/ipython-input-1770942647.py:14: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(data=df_results, x="Model", y=metric, ax=axes[i], palette=colors)
/tmp/ipython-input-1770942647.py:14: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(data=df_results, x="Model", y=metric, ax=axes[i], palette=colors)
/tmp/ipython-input-1770942647.py:14: FutureWarning: 

Passing `palette` without assigning `hue` is d